<a href="https://colab.research.google.com/github/DMahjoob/AvenuesConsultingProjects/blob/main/Avenues_Coffee_Chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
filename = files.upload()

KeyboardInterrupt: 

In [2]:
import pandas as pd
import io
# comment here as a tester
df = pd.read_csv("Responses.csv")
df = df.drop(['Timestamp'], axis=1)

In [3]:
memberList = []
num_rows = len(df)
row_index = 0

while row_index < num_rows:
  memberList.append([', '.join(map(str, df.iloc[row_index]))])
  row_index += 1

print(memberList)

[["Aaron Tom, Playing sports/working out, Exploring LA, Trying new food spots, Cookies and Cream, Strawberry, Butter Pecan, Time Travel, NY, What's a bedtime?, Indie, In a big city, Trying new restaurants, Winter, Taco stand, Doheny Library, 7"], ['Ahaana Naimpally, Playing sports/working out, Exploring LA, Trying new food spots, Cookies and Cream, Strawberry, Chocolate, Invisibility, NY, Early bird gets the worm, Pop, Somewhere on a beach, Trying new restaurants, Fall, Taco stand, Ministry of Coffee (MOC), 6'], ["Aidan van Dijk, Listening to music, Cooking/baking, Exploring LA, Coffee, Mint Chocolate Chip, Caramel, Time Travel, NY, What's a bedtime?, Rock, Somewhere on a beach, Trying new restaurants, Fall, Taco stand, In my own room, 7"], ["Alex Hymson, Watching movies/TV shows, Playing sports/working out, Exploring LA, Cookies and Cream, Coffee, Cookie Dough, Super Strength, NY, What's a bedtime?, Indie, Hiking/mountains, Exploring LA, Winter, Taco stand, Doheny Library, 2"], ['Anna

In [4]:
pip install sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Load the SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize variables to store the best match
taken_people = set()  # Set to track people already matched
matches = []  # List to store the final pairs

# Function to calculate the similarity of each field
def calculate_similarity(member, member2):
    # Extract the relevant fields (e.g., favorite food)
    field1 = member[0].split(',')[1].strip()  # Extract favorite food
    field2 = member2[0].split(',')[1].strip()  # Extract favorite food

    # Get sentence embeddings for each field
    embedding1 = model.encode([field1])
    embedding2 = model.encode([field2])

    # Calculate cosine similarity
    similarity_score = cosine_similarity(embedding1, embedding2)[0][0]

    return similarity_score

# Loop through each member in the list
for member in memberList:
    if member[0].split(',')[0] in taken_people:
        continue  # Skip if this person is already matched

    match_list = {}

    # Loop through each member in the list again to compare
    for member2 in memberList:
        if member == member2 or member2[0].split(',')[0] in taken_people:
            continue  # Skip if this person is already matched or if matching to self

        # Calculate the similarity for the specific field (e.g., favorite food)
        similarity_score = calculate_similarity(member, member2) + random.randint(-1,1)/10

        # Store the match with its similarity score
        match_list[similarity_score] = member2[0].split(',')[0]

    # Sort matches by similarity score (descending)
    sorted_scores = sorted(match_list.keys(), reverse=True)

    # Get the best match
    if sorted_scores:
        best_match_score = sorted_scores[0]
        matched_person = match_list[best_match_score]

        # Add both people to the taken_people set
        taken_people.add(member[0].split(',')[0])  # Add current person
        taken_people.add(matched_person)  # Add matched person

        # Store the match
        matches.append((member[0].split(',')[0], matched_person, best_match_score))

# Handle the case where there are unmatched members (if odd number)
unmatched_members = [m for m in memberList if m[0].split(',')[0] not in taken_people]

# If there's an odd number of members, match the last person with the next available one
if unmatched_members:
    for i in range(0, len(unmatched_members), 2):
        if i+1 < len(unmatched_members):  # Match in pairs if possible
            person1 = unmatched_members[i][0].split(',')[0]
            person2 = unmatched_members[i+1][0].split(',')[0]
            matches.append((person1, person2, 0.0))  # Assign a score of 0.0 for forced matches

# Print the matches
for match in matches:
    print(f"Best Match: {match[0]} and {match[1]} with a similarity score of {match[2]:.4f}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Best Match: Aaron Tom and Vicky Zhang with a similarity score of 1.1000
Best Match: Ahaana Naimpally and Valen Lu with a similarity score of 1.0000
Best Match: Aidan van Dijk and Thomas Bittencourt with a similarity score of 1.0000
Best Match: Alex Hymson and Fay Li with a similarity score of 1.1000
Best Match: Anna Chi and Jiho Kim with a similarity score of 1.1000
Best Match: Ariana Amiri and Sarina Mody with a similarity score of 1.1000
Best Match: Aseem Singru and Maxim Wu with a similarity score of 1.1000
Best Match: Assem Syrgabayeva and Philip Hu with a similarity score of 1.1000
Best Match: Ben Reed and Nathan Brigger with a similarity score of 1.1000
Best Match: Carter Woltz and Ryan Bohn with a similarity score of 1.1000
Best Match: Christian Kim and Natalie Wijaya with a similarity score of 1.1000
Best Match: Darius Mahjoob and Luke Stevens  with a similarity score of 1.1000
Best Match: Debra Erdenemandakh and Jeffrey Yang with a similarity score of 1.0000
Best Match: Elaine